# Download 1/2 degree plots

In [1]:
import ee, geemap
print(ee.__version__)
print(geemap.__version__)

0.1.221
0.6.13


In [2]:
ee.Initialize()

### Create cloud mask and NDRE functions

In [3]:
# Cloud mask function
def s2_cloud_mask(img):
    """Function using built-in bit mask to mask clouds/cirrus."""
    qa = img.select('QA60')
    
    cloud_mask = 1 << 10
    cirrus_mask = 1 << 11
    
    mask = (qa.bitwiseAnd(cloud_mask).eq(0)
            .And(qa.bitwiseAnd(cirrus_mask).eq(0))
           )
    
    return img.updateMask(mask).divide(10000)

In [4]:
# Calculate NDRE
def ndre_band(img):
    """Function to calculate ndre for an image."""
    ndre = img.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return img.addBands(ndre)

### Get region data and sentinel-2 data from GEE

In [5]:
# List sub-regions using GAUL polygon data
br = (ee.FeatureCollection("FAO/GAUL/2015/level1")
      .filterMetadata('ADM0_NAME', 'equals', 'Brazil')
      .aggregate_array('ADM1_NAME')
     )

print(br.getInfo())

['Alagoas', 'Bahia', 'Ceara', 'Espirito Santo', 'Maranhao', 'Minas Gerais', 'Paraiba', 'Pernambuco', 'Piaui', 'Rio De Janeiro', 'Rio Grande Do Norte', 'Sao Paulo', 'Sergipe', 'Name Unknown', 'Name Unknown', 'Name Unknown', 'Acre', 'Amapa', 'Amazonas', 'Distrito Federal', 'Goias', 'Mato Grosso', 'Mato Grosso Do Sul', 'Para', 'Parana', 'Rio Grande Do Sul', 'Rondonia', 'Roraima', 'Santa Catarina', 'Tocantins', 'Name Unknown']


In [6]:
# Get a sub-region from above to test grid function later
br = (ee.FeatureCollection("FAO/GAUL/2015/level1")
      .filterMetadata('ADM0_NAME', 'equals', 'Brazil')
      .filterMetadata('ADM1_NAME', 'equals', 'Espirito Santo')
     )

In [7]:
# Get sentinel-2 level-1C TOA data
s2_col = (ee.ImageCollection('COPERNICUS/S2_SR')
              .select(['B4', 'B3', 'B2', 'B5', 'B8', 'QA60'])
              .filterDate('2016-01-01', '2018-12-30')
              .sort('CLOUDY_PIXEL_PERCENTAGE', False)
              .map(s2_cloud_mask)
              .map(ndre_band)
              .filterBounds(br)
              .qualityMosaic('NDRE')
             )

In [8]:
# Just to note the crs
proj = s2_col.projection().getInfo()
print(proj)

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


## Create a grid

### Method 1 (failed)
This method will could probably work better than the second, but I couldn't get it perfectly translated from javascript.

In [9]:
# Method 1 -- Failed, ends up in the ocean and invisible
# https://code.earthengine.google.com/cf1d2e6e8c6375d33c48f0a121293bb0#
import math

lon_start = -60.00
lon_end = -60.50
lat_start = -2.35
lat_end = -2.85

num_cells = 4
lon_edge = (lon_end-lon_start)/ math.sqrt(num_cells)
lat_edge = (lat_end-lat_start)/ math.sqrt(num_cells)

polys = []
cell_id = 0

lon = lon_start
while lon < lon_end:
    x1 = lon
    x2 = lon + lon_edge
    lon += lon_edge
    
    lat = lat_start
    while lat < lat_end:
        cell_id = cell_id + 1
        y1 = lat
        y2 = lat + lat_edge
        lat += lat_edge
        
        ee.List(polys).add(ee.Feature(ee.Geometry.Rectangle(x1, y1, x2, y2), {'label': cell_id}))

grid = ee.FeatureCollection(polys)

### Method 2
Not 100% perfect. You have to have a reasonably-sized roi drawn. If it's too big, it's hard to predict where the grid will be drawn. But hey, at least it gives me some squares.

In [10]:
# Method 2: geemap.show_youtube('N7rK2aV1R4c')
# Create a map and draw a square interactively
Map = geemap.Map()
Map.addLayer(br)
Map.center_object(br, zoom=7)

palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', 
           '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']

Map.addLayer(s2_col.select('NDRE'), {'palette': palette}, 'NDRE')
#Map

In [11]:
# Set name
my_roi = br
#print(my_roi)

In [12]:
# Save feature
my_roi = ee.FeatureCollection(my_roi)

In [13]:
# Create grid
# https://developers.google.com/earth-engine/tutorials/community/drawing-tools

def make_grid(region, a_scale):
    """
    Creates a grid around a specified ROI.
    User inputs their reasonably small ROI.
    User inputs a scale where 100000 = 100km.
    """
    # Creates image with 2 bands ('longitude', 'latitude') in degrees
    lonLat = ee.Image.pixelLonLat()

    # Select bands, multiply times big number, and truncate
    lonGrid = (lonLat
               .select('latitude')
               .multiply(10000000)
               .toInt()
              )
    latGrid = (lonLat
              .select('longitude')
              .multiply(10000000)
              .toInt()
              )

    # Multiply lat and lon images and reduce to vectors
    grid = (lonGrid
            .multiply(latGrid)
            .reduceToVectors(
                geometry = region,
                scale = a_scale, # 100km-sized boxes needs 100,000
                geometryType = 'polygon')
           )
    
    return(grid)

In [14]:
# Make test grid and add to map
grid_55km = make_grid(br, 55000)

## Visualize selected grid tiles + images

In [15]:
# Access coordinates of grid squares
grid_dict = grid_55km.getInfo()

feats = grid_dict['features']
coord_list = []
for d in feats:
    geom = d['geometry']
    coords = geom['coordinates']
    coord_list.append(coords)

In [16]:
# Create a list of several ee.Geometry.Polygons
polys = []
for coord in coord_list:
    poly = ee.Geometry.Polygon(coord)
    polys.append(poly)

In [17]:
# Clip image to polygons
clipped_imgs = []
for poly in polys:
    ndre_img = s2_col.select(['NDRE'])
    clipped_img = ndre_img.clip(poly)
    clipped_imgs.append(clipped_img)

print(clipped_imgs)
print(len(clipped_imgs))

[<ee.image.Image object at 0x7fa09d1d1dd8>, <ee.image.Image object at 0x7fa09d1d1e80>, <ee.image.Image object at 0x7fa09d1d1f28>, <ee.image.Image object at 0x7fa09d1d1fd0>, <ee.image.Image object at 0x7fa09d1de0b8>, <ee.image.Image object at 0x7fa09d1de160>, <ee.image.Image object at 0x7fa09d1de208>, <ee.image.Image object at 0x7fa09d1de2b0>, <ee.image.Image object at 0x7fa09d1de358>, <ee.image.Image object at 0x7fa09d1de400>, <ee.image.Image object at 0x7fa09d1de4a8>, <ee.image.Image object at 0x7fa09d1de550>, <ee.image.Image object at 0x7fa09d1de5f8>, <ee.image.Image object at 0x7fa09d1de6a0>, <ee.image.Image object at 0x7fa09d1de748>, <ee.image.Image object at 0x7fa09d1de7f0>, <ee.image.Image object at 0x7fa09d1de898>, <ee.image.Image object at 0x7fa09d1de940>]
18


In [19]:
# Visualize a polygon or two
Map = geemap.Map()

palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', 
           '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']

Map.addLayer(polys)
Map.addLayer(clipped_imgs[0], {'palette': palette}, 'grid1')
Map.center_object(polys[0], zoom=7)
Map

AttributeError: 

The image argument in 'addLayer' function must be an instace of one of ee.Image, ee.Geometry, ee.Feature or ee.FeatureCollection.

## Export all grid tiles + img individually

In [44]:
for num in range(len(clipped_imgs)):
    filename = '/Users/morgansteckler/Desktop/test/img_{}.tif'.format(num)
    geemap.ee_export_image(clipped_imgs[num], filename=filename, 
                                scale=20, region=polys[num], file_per_band=False)

The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif
The filename must end with .tif


## Export grid as shapfile

In [40]:
# Set all ee.Geometry.Polygon to ee.Feature
feat_list = []
for poly in polys:
    feat = ee.Feature(poly)
    feat_list.append(feat)
feat_list

In [41]:
# Create collection from features
fromList = ee.FeatureCollection(feat_list)
fromList

In [46]:
# Export entire grid as shapefile
geemap.ee_to_shp(fromList, filename='/Users/morgansteckler/Desktop/tropical_phenology_gee/morgan/grid.shp')

Generating URL ...
Please wait ...
Data downloaded to /Users/morgansteckler/Desktop/tropical_phenology_gee/morgan/grid.shp
